** This is the explanation of how we can generate a permanent token; I have an admin access to the facebook page I am trying to scarpe the comments from
Steps I took are listed here:
1) Go to https://developers.facebook.com/ and make an APP
You need to generate a short lived user token for the app; in Graph API Explorer under the "User Token" bar find "Get User access token": the generated token is your short token which will expire in about an hour. 
2) Navigate in the main page (developer page) and under setting icon (on the left bar) click on basics: you can find your APP ID and App secret.
3) Run the code below, so that you will have a long lived token which has expiry day of about 2 months **

In [ ]:
# make a long lived token
import requests

app_id = 'your app id'
app_secret = 'your app secret'
user_short_token = 'your short token'
access_token_url = "https://graph.facebook.com/oauth/access_token?grant_type=fb_exchange_token&client_id={}&client_secret={}&fb_exchange_token={}".format(app_id, app_secret, user_short_token)

r = requests.get(access_token_url)

access_token_info = r.json()
user_long_token = access_token_info['access_token']

print(access_token_info)



**Now you need to have a permanent token: follow the next to cells **

In [ ]:
import facebook

graph = facebook.GraphAPI(access_token=user_long_token,version="2.8")

In [ ]:
# permanent token
pages_data = graph.get_object("/me/accounts")

print(pages_data)

**Running the above cell you will get access to data related to any page that you are its admin. For each page there exsists a page_id; for that page_id you can print the permanent page_token and save it for your self. So that you dont need to repeat the above commands any more!**

In [ ]:
page_id = 'your page id'
page_token = None

for item in pages_data['data']:
    if item['id'] == page_id:
        page_token = item['access_token']

# this is the permanent page token :D
print(page_token)

In [ ]:
graph = facebook.GraphAPI(access_token=page_token,version="2.8")

print(graph.get_object(id=page_id, fields='about, category'))



In [ ]:
page_impressions = graph.get_connections(id=page_id,
                                         connection_name='insights',
                                         metric='page_impressions',
                                         date_preset='yesterday',
                                         period='week',
                                         show_description_from_api_doc=True)
print(page_impressions)

In [ ]:
some_info = graph.get_object(id=page_id, fields='about, website')
print(some_info)

**Using "graph.get_object" you can get access to the last 25 posts which are posted in the facebook page; each post has an id, a message( which is the title of the post), and the posting time. 
Here, in a list (post_ids) I append all the post ids for my self. Later, I will use this list to access the comments realted to each post. I make another list to append all the titles (masseges) for each post. And, finally, I make a dictionary connecting each id to its raled title. **

In [ ]:
posts = graph.get_object(id=page_id, fields='posts')

# post_1 = posts['posts']['data'][0]
# post_2 = posts['posts']['data'][1]

In [ ]:
posts_ids = []
for i in range(len(posts['posts']['data'])):
    posts_ids.append(posts['posts']['data'][i]['id'])


In [ ]:
posts_msges = []
for i in range(len(posts['posts']['data'])):
    posts_msges.append(posts['posts']['data'][i]['message'])

In [ ]:
list_needed = dict(zip(posts_ids, posts_msges ))

**Now, for all posts (25 total post) I append the commnets into one large list. **

In [ ]:
list_new = []
for i in posts_id:
    list_new.append(graph.get_connections(id= i ,connection_name='comments', limit = 1000,include_hidden = True, order ='reverse_chronological'
                                        ,filter ='stream', summary ='total_count'))
    
   